In [33]:
import sympy as sp
import networkx as nx
import numpy as np
from math import factorial

import linearframework.graph_operations as g_ops
import linearframework.ca_recurrence as ca
import linearframework.linear_framework_results as lfr

## TODO

- [ ] instructions for how to quickly calculate many datapoints of a graph
- [ ] tests for generalized_aldous_schepp -- just that they are equal for erlang processes, and raises
- [ ] raises tests for k-th moment expression calculator
- [ ] splitting probability
- [ ] conditional mean first passage time


In [34]:
g_dict = {
    ('1', '2'): 1,
    ('1', '3'): 2,
    ('2', '1'): 3,
    ('2', '3'): 4,
    ('3', '1'): 5,
    ('3', '2'): 6
}

k3 = g_ops.dict_to_graph(g_dict)

k3_sym_lap = ca.graph_to_sym_laplacian(k3)

k3_sym_lap

Matrix([
[1_2 + 1_3,      -1_2,      -1_3],
[     -2_1, 2_1 + 2_3,      -2_3],
[     -3_1,      -3_2, 3_1 + 3_2]])

In [35]:
n = k3_sym_lap.rows
Q_n_minus_2 = ca.get_sigma_Q_k(k3_sym_lap, n-2)[1]

str(lfr.ca_kth_moment_numerator(k3,  k3_sym_lap, Q_n_minus_2, '1', '3', 1))

'1_2 + 2_1 + 2_3'

In [36]:
str(sp.simplify(lfr.ca_kth_moment_numerator(k3,  k3_sym_lap, Q_n_minus_2, '1', '3', 2)))

'2*1_2*2_1 + 2*1_2*(1_2 + 1_3) + 2*1_2*(2_1 + 2_3) + 2*(2_1 + 2_3)**2'

In [37]:
str(sp.simplify(lfr.ca_kth_moment_numerator(k3,  k3_sym_lap, Q_n_minus_2, '1', '3', 3)))

'6*1_2**2*2_1 + 6*1_2*2_1*(1_2 + 1_3) + 12*1_2*2_1*(2_1 + 2_3) + 6*1_2*(1_2 + 1_3)**2 + 6*1_2*(1_2 + 1_3)*(2_1 + 2_3) + 6*1_2*(2_1 + 2_3)**2 + 6*(2_1 + 2_3)**3'

In [38]:
erlang_weight = 10 ** (6 * np.random.rand() - 3)
e_5_dict = {
    ('1', '2'): erlang_weight,
    ('2', '3'): erlang_weight,
    ('3', '4'): erlang_weight,
    ('4', '5'): erlang_weight
}
e_5 = g_ops.dict_to_graph(e_5_dict)

e_5_sym_lap = ca.graph_to_sym_laplacian(e_5)

n = e_5_sym_lap.rows

Q_n_minus_2 = ca.get_sigma_Q_k(e_5_sym_lap, n - 2)[1]

In [49]:
type(lfr.ca_kth_moment_numerator(e_5, e_5_sym_lap, Q_n_minus_2, '1', '5', 2))

sympy.core.add.Add

In [44]:
def generalized_randomness_parameter(graph,  Lap, Q_n_minus_2, source, target, moment):
    """calculates the symbolic expression of the moment-th moment of the graph over the mean of the graph to the power of moment

    Args:
        graph (nx.DiGraph): networkx graph of interent
        Lap (sympy.matrices.dense.MutableDenseMatrix): symbolic laplacian of graph
        Q_n_minus_2 (sympy.matrices.dense.MutableDenseMatrix): Q_(n-2) matrix found using the CA recurrence
        source (str): id of source vertex
        target (str): id of target vertex
        moment (int): moment of interest

    Returns:
        sympy.core.add.Add: symbolic expression of the moment-th moment of the graph over the mean of the graph to the power of moment
    """
    numerator = lfr.ca_kth_moment_numerator(graph,  Lap, Q_n_minus_2, source, target, moment)
    denominator = lfr.ca_kth_moment_numerator(graph,  Lap, Q_n_minus_2, source, target, 1) ** moment
    return numerator / denominator

s_2 = generalized_randomness_parameter(e_5, e_5_sym_lap, Q_n_minus_2, '1', '5', 2)

free_symbols = s_2.free_symbols

e_r = sp.symbols('e_r')

symbol_dict = {}
for free_symbol in free_symbols:
    symbol_dict[free_symbol] = e_r

s_2.subs(symbol_dict)

5/4

In [48]:
def guzman_alca_equation(n, m):
    """
    calculates the m-th generalized randomness parameter of an erlang process on n vertices

    Args:
        n (int): the number of vertices in the erlang process.
        m (int): the moment in the numerator, and the exponent in the denominator of the generalized randomness parameter.

    Returns:
        float: the m-th generalized randomness parameter of an erlang process on n vertices
    """
    numerator = factorial(n + m - 2)

    denominator = factorial(n - 2)
    denominator *= (n-1)**m
    return numerator / denominator

guzman_alca_equation(5, 2)

1.25

In [55]:
def gen_erlang_process_dict(number_of_states, rate=None):
    """generates the edge-to-weight dictionary of an erlang process. 
    The uniform rate can be pre-set, or generated randomly

    Args:
        number_of_states (int): number of states
        rate (float, optional): uniform rate. Defaults to None.

    Returns:
        dict[tuple[str]: float]: edge-to-weight dictionary of an erlang process
    """
    if rate == None:
        rate = 10 ** (6 * np.random.rand() - 3)
    
    erlang_dict = {}
    for i in range(1, (number_of_states + 1)):
        erlang_dict[(f'{i}', f'{i + 1}')] = rate

    return erlang_dict

gen_erlang_process_dict(5, rate=1)

{('1', '2'): 1, ('2', '3'): 1, ('3', '4'): 1, ('4', '5'): 1, ('5', '6'): 1}